In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType, VectorizedQuery,QueryAnswerType, QueryCaptionType
from openai import AzureOpenAI

service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
wiki_index = os.getenv("AZURE_SEARCH_INDEX_WIKI")
video_index = os.getenv("AZURE_SEARCH_INDEX_VIDEO")
api_version = os.environ.get("AZURE_OPENAI_PREVIEW_API_VERSION")
aoai_api_key = os.environ.get("AZURE_OPENAI_KEY")
embedding_model = os.environ.get("AZURE_OPENAI_EMBEDDING_NAME")
chat_model = os.environ.get("AZURE_OPENAI_MODEL")
credential = DefaultAzureCredential()

ad_token_provider = get_bearer_token_provider(credential,"https://cognitiveservices.azure.com/.default")
azure_endpoint = (
            os.environ.get("AZURE_OPENAI_ENDPOINT")
            if os.environ.get("AZURE_OPENAI_ENDPOINT")
            else f"https://{os.environ.get('AZURE_OPENAI_RESOURCE')}.openai.azure.com/")
default_headers = {"x-ms-useragent": "GitHubSampleWebApp/AsyncAzureOpenAI/1.0.0"}

wiki_search_client = SearchClient(service_endpoint, wiki_index, credential)
video_search_client = SearchClient(service_endpoint, video_index, credential)

openai_client = AzureOpenAI(
  api_version = api_version, 
  api_key  = aoai_api_key,
  azure_ad_token_provider = ad_token_provider,
  azure_endpoint = azure_endpoint,
  default_headers = default_headers)


In [3]:
def generate_embeddings(query, model):
    # Generate embeddings for the provided text using the specified model
    embeddings_response = openai_client.embeddings.create(model=model, input=query)
    # Extract the embedding data from the response
    embedding = embeddings_response.data[0].embedding
    return embedding


In [31]:
query = "how to get cad into your workspace"
# query = "Registering a PDM Link Server"
vector_query = VectorizedQuery(vector=generate_embeddings(query, embedding_model), 
                               k_nearest_neighbors=3, fields="text_vector",)

results_wiki = list(wiki_search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    select=["title", "chunk", "url_metadata"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="semantic",
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3,
))


In [32]:
results_video = list(video_search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    select=["title", "chunk", "url_metadata"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="semantic",
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3,
))

In [33]:
# semantic_answers = results_wiki.get_answers()
# for answer in semantic_answers:
#     if answer.highlights:
#         print(f"Semantic Answer: {answer.highlights}")
#     else:
#         print(f"Semantic Answer: {answer.text}")
#     print(f"Semantic Answer Score: {answer.score}\n")

# for result in results_wiki:
#     print(f"Title: {result['title']}")
#     print(f"Reranker Score: {result['@search.reranker_score']}")
#     print(f"URL: {result['url_metadata']}")
#     captions = result["@search.captions"]
#     if captions:
#         caption = captions[0]
#         if caption.highlights:
#             print(f"Caption: {caption.highlights}\n")
#         else:
#             print(f"Caption: {caption.text}\n")

In [34]:
# results_wiki = list(wiki_search_client.search(search_text=None, vector_queries= [query_embbeding], semantic_configuration_name='semantic', top=3, query_type=QueryType.SEMANTIC))
# results_video = list(video_search_client.search(search_text=None,vector_queries= [query_embbeding],semantic_configuration_name='semantic', top=3, query_type=QueryType.SEMANTIC))

In [35]:
results = results_wiki + results_video
filter_result = []
for d in results:
    if d in results_wiki:
        d['container'] = 'wiki'
    elif d in results_video:
        d['container'] = 'video'

sorted_data = sorted(results_wiki + results_video, key=lambda x: x["@search.reranker_score"], reverse=True)
selected_container = sorted_data[0]['container']
selected_chunks = []
for i in sorted_data:
    if i['container'] == selected_container and len(selected_chunks)<2:
        selected_chunks.append(i)


context_str = f"""
**documents: 1**

{selected_chunks[0]['chunk']}


**documents: 2**

{selected_chunks[1]['chunk']}
"""

if selected_container == "video":
    citation_prompt = "Context is a transcript which has timestamp in the form HH:MM:SS in each line above texts. Once you provide the answer, include the all the citations based on each of the timestamp and documents number [timestap, document number] at the end which are used to generate the answer. citation format is [['00:11:00', 1], ['00:1:44', 2]]"
else:
    citation_prompt = "Context is provided in the form of multiple documents. Once you provide the answer, include the citation based on the documents number at the end. citation format is [1,2]"

RAG_SYSTEM_PROMPT = f"""\
Context information is below.
---------------------
{context_str}
---------------------
INSTRUCTIONS:
1. You are an assistant who helps users answer their queries.
2. Answer the user's question from the above Context.
3. Give answer in step by step format.
4. Keep your answer solely on the information given in the Context above.
5. {citation_prompt}
6. Do not create or derive your own answer. If the answer is not directly available in the context, just reply stating, 'There is no answer available'
"""


In [36]:
import instructor
from pydantic import BaseModel, Field
from typing import List, Dict, Optional


class Asnwer_Cittion(BaseModel):
    """ 
    Asnwer and Citations
    """
    if selected_container == 'video':
        citation: List[List] = Field(
            description="citations")
    else:
        citation: List[int] = Field(
            description="citations")
        
    answer: str = Field(description="Answer of usr's question, do not include citations in answer")
    


instructor_client = instructor.from_openai(openai_client)

# Extract structured data from natural language
user_info = instructor_client.chat.completions.create(
    model=chat_model,
    response_model=Asnwer_Cittion,
    messages=[{"role": "system", "content": RAG_SYSTEM_PROMPT},
              {"role": "user", "content": query}],
)


In [37]:
query

'how to get cad into your workspace'

In [38]:
print(user_info.answer)

To get CAD into your workspace, follow these steps:

1. **Check Out and Add to Workspace**: Locate the part you want to work on and check it out. When you check out a part, it means you have control over it, and no other user can modify it at the same time. This ensures that multiple users aren't working on the same part simultaneously. After checking out, add it to your workspace. [00:30:25, 1] [00:31:09, 1]

2. **Alternative Method for Read-Only Users**: For read-only users or if you don't need to modify the part, use the 'Open in Creole' method. In the browser view, find the part you want and click on the square icon next to its file name column that says 'Open in Creole'. This action will add the part directly to your workspace. [00:37:59, 2] [00:38:30, 2] [00:38:51, 2]


In [39]:
print(user_info.citation)

[['00:30:25', 1], ['00:31:09', 1], ['00:37:59', 2], ['00:38:30', 2], ['00:38:51', 2]]


In [30]:
selected_chunks[1]

{'url_metadata': 'https://microsoft.sharepoint.com/:w:/t/collearninganddevelopmentlnd/EVb_PRHztPBFlkz5OHdKSNcB0qdvUuqnyvoaFnmmjw7dFQ?e=qZ2Dav',
 'title': 'PDM_Training_02-Connecting_to_Server.docx',
 'chunk': 'PDM Link Training Document #02\nConnecting to the PDM Link Server\n\nThese are the steps for a new user to access the PDM Link server for the first time:\n\n\n1) Launch Pro/E\n\n\n2) Select the drop down menu FILE + MANAGE SESSION + SERVER MANAGEMENT as shown below:\n(In Wildfire 5 use TOOLS + SERVER MANAGER) \n\n\nThis will open the “Server Manager” window.\n\n\n3) Select the drop down menu SERVER + REGISTER NEW SERVER as shown below:\n\n\n\n\n\n4) This will open the “Register New Server” window. Enter a name for the server (this can be anything, but “PDM Link” would be appropriate). Enter the location ( http://ptcwindchill/Windchill ) as shown below:\n\n\n\nThen select the “CHECK” box.\n\n\n5) Now, PDM Link (or whatever else you called it) should show up in the Server Manager w

In [20]:
print(RAG_SYSTEM_PROMPT)

Context information is below.
---------------------

**documents: 1**

working on that part. I can expect changes to be checked back in, maybe by that user in the near future.
00:30:21 Speaker 1
So check out and add to workspaces.
00:30:23 Speaker 1
What this page?
00:30:24 Speaker 1
Is we talked about all those same.
00:30:25 Speaker 1
Details I'm going to say OK.
00:30:30 Speaker 1
And now my workspace has that same copy of the same part, but it has a new status icon.
00:30:36 Speaker 6
If I can mouse over it.
00:30:38 Speaker 1
It says checked out by you. That means my username has control of this object and no other user in wind chill right now could go check it out at the same time.
00:30:49 Speaker 1
That's how we throttle a ton of users across a bunch of different countries and time zones actively working in the same.
00:30:56 Speaker 1
CAD database at the same.
00:30:58 Speaker 1
Time. If you want to change a part and you have read write access to it, your first step should be 

In [14]:
!azd auth login

Retrieving subscriptions...
Logged in to Azure.



To update to the latest version, run:
winget upgrade Microsoft.Azd


In [13]:
from langchain_openai import AzureChatOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from backend.auth.auth_utils import get_authenticated_user_details
from backend.security.ms_defender_utils import get_msdefender_user_json
from quart import request
# from llama_index.llms.azure_openai import AzureOpenAI

In [14]:
api_version = os.environ.get("AZURE_OPENAI_PREVIEW_API_VERSION")
aoai_api_key = os.environ.get("AZURE_OPENAI_KEY")
ad_token_provider = get_bearer_token_provider(DefaultAzureCredential(),"https://cognitiveservices.azure.com/.default")
endpoint = (
            os.environ.get("AZURE_OPENAI_ENDPOINT")
            if os.environ.get("AZURE_OPENAI_ENDPOINT")
            else f"https://{os.environ.get('AZURE_OPENAI_RESOURCE')}.openai.azure.com/"
        )
default_headers = {"x-ms-useragent": "GitHubSampleWebApp/AsyncAzureOpenAI/1.0.0"}


authenticated_user_details = get_authenticated_user_details({})
conversation_id = None     
user_json = get_msdefender_user_json(authenticated_user_details, {}, conversation_id)


# from llama_index.llms.azure_openai import AzureOpenAI

# llm = AzureOpenAI(
#     engine=os.environ.get("AZURE_OPENAI_MODEL"),
#     model=os.environ.get("AZURE_OPENAI_MODEL_NAME"),
#     api_key=aoai_api_key,
#     azure_endpoint=endpoint,
#     api_version=api_version,
#     AZURE_AD_TOKEN_PROVIDER = ad_token_provider,
#     use_azure_ad =True,
#     kwargs={'default_headers': default_headers, "user": user_json},
# )

llm = AzureChatOpenAI(
    deployment_name = os.environ.get("AZURE_OPENAI_MODEL"),
    openai_api_version = api_version,
    openai_api_key = aoai_api_key,
    azure_ad_token_provider = ad_token_provider,
    default_headers = default_headers,
    azure_endpoint = endpoint,
    include_response_headers = True,
    streaming= True,
    temperature= 0.15,
    # user_json = user_json
    # model_kwargs = {'': user_json}
    # logprobs=True,
    model_kwargs={"stream_options": {"include_usage": True}}
    )

In [45]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with colorful personality."
    ),
    ChatMessage(role="user", content="Hello"),
]

response = llm.chat(messages)
print(response)

assistant: Ahoy there, matey! Welcome aboard! What brings ye to these treacherous waters today? Lookin' for treasure, a tale, or perhaps a bit o' both? Arrr!


In [6]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
messages = [
    (
        "system",
        "You are a helpful assistant that helps user.",
    ),
    ("human", "How are you?"),
]
ai_msg = llm.invoke(messages)
# apim_request_id = ai_msg.response_metadata['headers']['apim-request-id']
ai_msg


AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_80a1bad4c7'}, id='run-3f724057-5589-4b85-9fd8-311050c8c8b3-0', usage_metadata={'input_tokens': 24, 'output_tokens': 28, 'total_tokens': 52})

In [40]:
from backend.utils import (
    format_as_ndjson,
    format_stream_response,
    format_non_streaming_response,
    convert_to_pf_format,
    format_pf_non_streaming_response,
)